In [1]:
import pyspark
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
usuarios = [
    (1, "Ailen", "Argentina", "20160413"),
    (2, "Alejandra", "Argentina", "20160512"),
    (3, "Paul", "Francia", "20150521"),
    (4, "William", "Estados Unidos", "20140305"),
]

usuariosRDD = sc.parallelize(usuarios);

publicaciones = [
    (30, 1, 20, "#Me #Photo #aburrida #Calor #hot #nodoymas qué calor", "20180412"),
    (22, 2, 5, "#Hola #Photo #aburrida #nodoymas", "20190101"),
    (25, 1, 1, "#Hola #Photo #friday #quepedo #music", "20190101"),
    (21, 3, 2, "#Photo Me", "20190202"),
    (12, 4, 4, "#hi hola", "20180512"),
]

publicacionesRDD = sc.parallelize(publicaciones);

### Encontrar el top10 de hashtags que más aparecen y que fueron publicados por usuarios de Argentina

Instagram almacena en un RDD (id_user, nickname,pais,fecha_alta) y en otro RDD (id_publicacion, id_user, comentario, id_foto, fecha)

In [42]:
hashtag_usuario = publicacionesRDD.map(lambda x: (x[1], x[3].split()))\
                    .flatMapValues(lambda x: x)\
                    .filter(lambda x: x[1].startswith("#"))

In [43]:
hashtag_usuario.collect()

[(1, '#Me'),
 (1, '#Photo'),
 (1, '#aburrida'),
 (1, '#Calor'),
 (1, '#hot'),
 (1, '#nodoymas'),
 (2, '#Hola'),
 (2, '#Photo'),
 (2, '#aburrida'),
 (2, '#nodoymas'),
 (1, '#Hola'),
 (1, '#Photo'),
 (1, '#friday'),
 (1, '#quepedo'),
 (1, '#music'),
 (3, '#Photo'),
 (4, '#hi')]

In [44]:
hastag_paises = usuariosRDD.map(lambda x: (x[0], x[2]))\
                    .join(hashtag_usuario)\
                    .map(lambda x: (x[1][1], [x[1][0]]))\
                    .reduceByKey(lambda x, y: x +y )
hastag_paises.collect()

[('#hot', ['Argentina']),
 ('#aburrida', ['Argentina', 'Argentina']),
 ('#Calor', ['Argentina']),
 ('#Photo', ['Argentina', 'Argentina', 'Argentina', 'Francia']),
 ('#friday', ['Argentina']),
 ('#Hola', ['Argentina', 'Argentina']),
 ('#hi', ['Estados Unidos']),
 ('#Me', ['Argentina']),
 ('#nodoymas', ['Argentina', 'Argentina']),
 ('#music', ['Argentina']),
 ('#quepedo', ['Argentina'])]

In [35]:
def filter_argentina(x):
    for element in x:
        if element != "Argentina":
            return False
    return True

In [45]:
hashtag_populars_arg = hastag_paises.filter(lambda x: filter_argentina(x[1]))\
            .map(lambda x: (x[0], len(x[1])))\
            .takeOrdered(10, lambda x: -x[1])
hashtag_populars_arg

[('#aburrida', 2),
 ('#Hola', 2),
 ('#nodoymas', 2),
 ('#hot', 1),
 ('#Calor', 1),
 ('#friday', 1),
 ('#Me', 1),
 ('#music', 1),
 ('#quepedo', 1)]

In [41]:
[ a[0] for a in pu3]

['#aburrida',
 '#Hola',
 '#nodoymas',
 '#hot',
 '#Calor',
 '#friday',
 '#Me',
 '#music',
 '#quepedo']

In [122]:
def filter_dif(listi):
    leni = len(listi)
    comun = []
    for i in range(leni):
        for j in range (i+1, leni):
            comun.append((listi[i], listi[j]))
    return comun

# def getCombinations(seq):
#     combinations = list()
#     for i in range(0,len(seq)):
#         for j in range(i+1,len(seq)):
#             combinations.append([seq[i],seq[j]])
#     return combinations

In [63]:
from itertools import permutations

In [125]:
usurios_hashtag_comun = hashtag_usuario.map(lambda x: (x[1], [x[0]]))\
                                        .reduceByKey(lambda x, y: x+y)\
                                        .map(lambda x: sorted(x[1]))\
                                        .filter(lambda x: len(x)>1)\
                                        .map(lambda x: filter_dif(x))\
                                        .flatMap(lambda x: x).distinct()
            
usurios_hashtag_comun.collect()

[(1, 1), (1, 2), (2, 3), (1, 3)]

In [104]:
usurios_hashtag_comun.map(lambda x: list(permutations(x, 2)))\
                        .flatMap(lambda x: x)\
                        .map(lambda x: sorted(x))\
                        .collect()

[[1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 1],
 [1, 3],
 [1, 2],
 [1, 2],
 [2, 3],
 [1, 1],
 [1, 2],
 [1, 3],
 [1, 3],
 [2, 3],
 [1, 3]]

In [87]:
a = [2,1].sort()
a